<a href="https://colab.research.google.com/github/seunghuilee91/Algorithm/blob/master/VAE(Variable_Autoencoder).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")

## Generative Model (생성모델)

입력 변수(latent variable) z로부터 결과물 x (가장 흔하게는 image)을 만들어내는 모델

> <img src="https://jamiekang.github.io/media/2017-05-21-auto-encoding-variational-bayes-generative-model.jpg">

> ## **생성모델의 분류**
> <img src="https://1.bp.blogspot.com/-0asYZ9AJH-M/WOjCRLXyz7I/AAAAAAAABmI/8E36K71xD0kjSVjDWajfz-EUJHl8LqGSQCK4B/s1600/vae_1.png">

## Autoencoder
- 입력 데이터 x 자신을 다시 만들어내려는 neural network 모델
- 구조는latent code z를 만드는 encoder와 x'를 만드는 decoder가 맞붙어 있는 형태
- 입력 x와 출력 x' 사이의 L2 loss (= mean squared error)를 최소화하도록 training
> <img src="https://jamiekang.github.io/media/2017-05-21-auto-encoding-variational-bayes-sae.jpg">


# VAE(Variable Autoencoder)
---
VAE는 Autoencoder의 특성을 물려 받았지만, 약간 다른 점이 있습니다.

- Autoencoder에서는 z가 training data와 특별히 관련이 없이 단순히 계산 중간에 나오는 deterministic한 값일 뿐입니다. 반면, VAE에서는 latent variable z가 continuous한 분포를 가지는 random variable이라는 점이 중요한 차이입니다. 이 latent variable z의 분포는 training 과정에서 data로부터 학습됩니다.

- VAE는 decoder 부분만 떼어내 Generative Model로 사용할 수 있습니다. Encoder는 decoder에 어떤 입력 z를 넣을지 학습으로 알게 됩니다.

- **latent variable z가 평균과 표준편차로 결정되는 확률 분포**를 가지는 것입니다.  VAE의 encoder는 주어진 x로 부터 z를 얻을 확률 p(z|x)로, VAE의 decoder는 z로부터 x를 얻을 확률 p(x|z)로 나타낼 수 있습니다.

> ## [VAE 아키텍처]
> <img src="https://jamiekang.github.io/media/2017-05-21-auto-encoding-variational-bayes-kvfrans-vae.jpg">
`수많은 고양이 사진이 있다고 칩시다. 사람은 고양이 사진들이 저마다 다르게 생겼다 하더라도 이들 사진이 고양이임을 단박에 알아낼 수 있습니다. 사람들은 고양이 사진을 픽셀 단위로 자세하게 보고 고양이라고 판단하는게 아니라, 털 색깔, 눈 모양, 이빨 개수 등 추상화된 특징을 보고 고양이라는 결론을 냅니다, 이를 잠재변수 z와 VAE 아키텍처 관점에서 이해해 보자면, encoder는 입력 데이터를 추상화하여 잠재적인 특징을 추출하는 역할, decoder는 이러한 잠재적인 특징을 바탕으로 원 데이터로 복원하는 역할을 한다고 해석해볼 수 있겠습니다. 실제로 잘 학습된 VAE는 임의의 z값을 decoder에 넣으면 다양한 데이터를 생성할 수 있다고 합니다.`


## Encoder
- Encoder의 역할은 데이터가 주어졌을 때 **Decoder가 원래의 데이터로 잘 복원할 수 있는  z 를 샘플링 할 수 있는 이상적인 확률분포  p(z|x)** 를 찾는 것
- 즉, encoder는 주어진 x로부터 z를 얻는 neural network
- 하지만 어떤 것이 이상적인 확률분포  p(z|x)  인지는 아무도 모른다. VAE 방법론에서는 이 문제를 해결하기 위해 **Variational inference(변분추론)**를 사용한다.

> ## **Variational inference**
- Variational inference의 기본 아이디어는 우리가 posterior inference를 어떻게 할 지 알고 있는 모델 Qϕ를 가지고 inference를 하되 parameter ϕ를 잘 조정해서 P에 최대한 가깝게 만들자는 것
- 이 때, 두 분포가 "가깝다"는 것을 어떻게 표현하기 위해 두 분포의 차이를 계산해주는 Kullback Leibler divergence가 사용됨
> <img src="https://i.imgur.com/05LNq0o.png">


> ## **The reparameterization trick**
>  *미분이 불가능한 샘플링을 미분이 가능하게 교묘하게 우회하는 방법* <br>
><br>
> Decoder에 latent variablez를 넣으려면 z가 random variable이므로 sampling을 해야하는데, sampling은 미분 가능하지가 않아서 gradient를 구할 수 없다.
> Back propagation은 편미분을 구함으로써 Gradient를 구하는 것인데,  z 를 확률분포에서 그냥 샘플링 한다면 체인룰이 중간에 끊기게 된다. 여기서는 이를 극복하기 위해서 Reparameterization trick을 사용했다.
> Reparameterization trick이란, 가우시안 정규 분포의 샘플을 추출하고 싶을 때, 아래의 식과 같이 샘플링을 하는 것을 말한다. 이렇게 샘플을 추출 하더라도 원래의 확률적 특성을 보존 한다. 그리고 이렇게 샘플링 하면  z 는 확률분포의 모수인 분산과 평균이 더해진 형태이므로 Back propagation 또한 가능하다.

> <img src="https://jamiekang.github.io/media/2017-05-21-auto-encoding-variational-bayes-kingma-pp18.jpg">
> <img src="https://jamiekang.github.io/media/2017-05-21-auto-encoding-variational-bayes-kingma-pp19.jpg">

## Decoder
- Decoder는 추출한 샘플을 입력으로 받아, 다시 원본으로 재구축하는 역할을 수행한다.
- latent variable z의 sample 값이 달라짐에 따라 아래 그림처럼 연속적으로 변하는 결과물을 만들어낼 수 있다. (= Variational이라고 불리는 이유)

## VAE의 장단점
- 장점
  - Log-likelihood를 통해 model의 quality를 평가할 수 있다.
  - 빠르고 training이 쉽다.
  - Loss 값을 계산할 수 있다.

- 단점
  - 결과물의 quality가 떨어진다(blurry하다).
  - q가 optimal하더라도 p와 같지 않을 수도 있다.


### [참고자료]

VAEs store latent attributes as probability distributions
<img src="https://miro.medium.com/max/1050/1*jkkHAb_IP4cjFPldb-u0GQ.png">

<img src="https://www.programmersought.com/images/766/7bc5256c1d78b58987845843c1e4755e.png>
---

<img src="https://miro.medium.com/max/1050/1*2IXeCfGWMw48EXmDdEgcLw.png">

<img src="https://www.programmersought.com/images/581/ac229b81aac60e710a602948bc562925.png">


 왜 GAN이 보다 Sharp한 이미지를 생성할 수 있는지에 대한 논의도 여전히 진행 중임

# 참고
https://jamiekang.github.io/2017/05/21/auto-encoding-variational-bayes/

https://ratsgo.github.io/generative%20model/2017/12/19/vi/

http://jaejunyoo.blogspot.com/2017/05/auto-encoding-variational-bayes-vae-3.html